In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

# SYDE 522 Final Project

## By Anirudh Kilambi and Youngwoo Lee

The purpose of this project is to apply techniques used in class to detect and identify faults and defects found in sewer pipelines. To accomplish this, we used training data found from the 2022 VideoPipe challenge for Real-World Video Understanding for Urban Pipe Inspection. For this project, we investigated the effectiveness of Convolutional Neural Networks (CNN) and Support Vector Machines (SVM) in order to come to an experimental conclusion on which model is most effective for this task.

## Part 1: Convolutional Neural Networks 